In [ ]:
!pip install scikit-multilearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 1.6 MB/s eta 0:00:00


In [ ]:
!pip install xgboost

In [ ]:
from google.colab import drive
import pandas as pd
from xgboost import XGBClassifier
from skmultilearn.problem_transform import ClassifierChain
from sklearn.model_selection import train_test_split
from sklearn.metrics import (classification_report, recall_score, hamming_loss, jaccard_score,
                             multilabel_confusion_matrix, f1_score, precision_score, accuracy_score,
                             roc_curve, auc,zero_one_loss)
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import make_pipeline
from sklearn.multioutput import MultiOutputClassifier
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import LabelPowerset
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset
file_path = 'Tactic_Dataset.csv'
df = pd.read_csv(file_path)

In [ ]:
# Drop the column 'Lateral Movement' from the DataFrame
df = df.drop(columns=['Lateral Movement'])

# Verify the column has been removed
print(df.columns)

Index(['Hash Name', 'com.mp4.videodownloader.receiver.restartservicereceiver',
       'com.dktools.liteforfb.mainactivity',
       'com.google.android.libraries.cast.companionlibrary.remotecontrol.videointentreceiver',
       'com.example.myapplicationtest.composesmsactivity',
       'com.sangcall.kchtmlactivity', 'ir.ali.korosh.hakh.mainactivity',
       'whuxyrlapwrkoxmzamujn.tpgbbryjtpo.ooegchzjunneeobe.wzymtmufclabfoxrtdhdq',
       'sun.photoalbum1.sunservice.sun9.send.whatsupdoc.llll1j1',
       'ehioinw.kjnt.dqpkcxd.qegehdservice',
       ...
       'Defence Evasion', 'Discovery', 'Execution', 'Initial Access',
       'Persistance', 'Exfiltration', 'Privilage Escalation',
       'Command and control', 'Collection', 'Impact'],
      dtype='object', length=13336)


In [ ]:
n_labels = 11  # replace with the actual number of label columns

In [ ]:
# Separate features and labels
X = df.iloc[:, 1:-n_labels]
y = df.iloc[:, -n_labels:]

In [ ]:
# Sum of occurrences of '1' in each label
label_sums = y.sum(axis=0)
print("Sum of occurrences of '1' in each label:\n", label_sums)

Sum of occurrences of '1' in each label:
 Credential Access        437
Defence Evasion         2464
Discovery               2190
Execution               1822
Initial Access            20
Persistance             1075
Exfiltration              95
Privilage Escalation     984
Command and control     2188
Collection              2431
Impact                   691
dtype: int64


In [ ]:
# Get label names
label_names = y.columns.tolist()
label_names

['Credential Access',
 'Defence Evasion',
 'Discovery',
 'Execution',
 'Initial Access',
 'Persistance',
 'Exfiltration',
 'Privilage Escalation',
 'Command and control',
 'Collection',
 'Impact']

In [ ]:
# Display the shape of the features and labels
print(f"Features shape: {X.shape}")
print(f"Labels shape: {y.shape}")

Features shape: (2774, 13324)
Labels shape: (2774, 11)


In [ ]:

# Evaluate the models
def evaluate_model(y_test, predictions, model_name, file, random_seed):
    file.write(f"\n\n********** Random Seed: {random_seed} **********\n\n")
    file.write(f"\nEvaluating {model_name}")
    file.write(f"\nAccuracy: {accuracy_score(y_test, predictions)}")
    file.write(f"\nMacro F1 Score: {f1_score(y_test, predictions, average='macro')}")
    file.write(f"\nWeighted F1 score: {f1_score(y_test, predictions, average='weighted')}")
    file.write(f"\nMicro F1 score: {f1_score(y_test, predictions, average='micro')}")
    file.write(f"\nMacro Precision: {precision_score(y_test, predictions, average='macro')}")
    file.write(f"\nWeighted Precision: {precision_score(y_test, predictions, average='weighted')}")
    file.write(f"\nMicro Precision: {precision_score(y_test, predictions, average='micro')}")
    file.write(f"\nMacro Recall: {recall_score(y_test, predictions, average='macro')}")
    file.write(f"\nWeighted Recall: {recall_score(y_test, predictions, average='weighted')}")
    file.write(f"\nMicro Recall: {recall_score(y_test, predictions, average='micro')}")
    file.write(f"\nHamming Loss: {hamming_loss(y_test, predictions)}")
    file.write(f"\nZero One Loss: {zero_one_loss(y_test, predictions)}")
    file.write(f"\nJaccard Similarity: {jaccard_score(y_test, predictions, average='samples')}")
    file.write(f"\nClassification Report:\n{classification_report(y_test, predictions,target_names=label_names)}")
    file.write(f"\nMultilabel Confusion Matrix:\n{multilabel_confusion_matrix(y_test, predictions)}")
    file.write("\n")

    print(f"\n\n********** Random Seed: {random_seed} **********\n\n")
    print(f"Evaluating {model_name}")
    print(f"Accuracy: {accuracy_score(y_test, predictions)}")
    print(f"Macro F1 Score: {f1_score(y_test, predictions, average='macro')}")
    print(f"Weighted F1 score: {f1_score(y_test, predictions, average='weighted')}")
    print(f"Micro F1 score: {f1_score(y_test, predictions, average='micro')}")
    print(f"Macro Precision: {precision_score(y_test, predictions, average='macro')}")
    print(f"Weighted Precision: {precision_score(y_test, predictions, average='weighted')}")
    print(f"Micro Precision: {precision_score(y_test, predictions, average='micro')}")
    print(f"Macro Recall: {recall_score(y_test, predictions, average='macro')}")
    print(f"Weighted Recall: {recall_score(y_test, predictions, average='weighted')}")
    print(f"Micro Recall: {recall_score(y_test, predictions, average='micro')}")
    print(f"Hamming Loss: {hamming_loss(y_test, predictions)}")
    print(f"Zero One Loss: {zero_one_loss(y_test, predictions)}")
    print(f"Jaccard Similarity: {jaccard_score(y_test, predictions, average='samples')}")
    print(f"Classification Report:\n{classification_report(y_test, predictions,target_names=label_names)}")
    print(f"Multilabel Confusion Matrix:\n{multilabel_confusion_matrix(y_test, predictions)}")
    print("\n")

In [ ]:
# Define different random seeds to evaluate
random_seeds = [42, 1433, 2396, 451, 995, 98, 262, 354, 560, 1600]

In [ ]:
# Open a single file to write all evaluations
with open("XGBoost_model_evaluations.txt", "w") as file:

    for seed in random_seeds:


        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        # Binary Relevance
        br_model = BinaryRelevance(XGBClassifier(gamma=0.632,learning_rate=0.242,max_depth=9,n_estimators=82,random_state=42))
        br_model.fit(X_train, y_train)
        br_predictions = br_model.predict(X_test)

        # initialize classifier chains ClassifierChain classifier
        chain_model = ClassifierChain(XGBClassifier(gamma=0.644,learning_rate=0.243,max_depth=5,n_estimators=191,random_state=42))
        # Training logistic regression model on train data
        chain_model.fit(X_train, y_train)
        # predict
        chain_predictions = chain_model.predict(X_test)

        # Label Powerset
        # initialize label powerset LabelPowerset classifier
        lb_model = LabelPowerset(XGBClassifier(gamma=0,learning_rate=0.3,max_depth=6,n_estimators=100,random_state=42))
        # train
        lb_model.fit(X_train, y_train)
        # predict
        lb_predictions = lb_model.predict(X_test)


        evaluate_model(y_test, br_predictions, "Binary Relevance", file, seed)
        evaluate_model(y_test, chain_predictions, "Classifier Chain", file, seed)
        evaluate_model(y_test, lb_predictions, "Label Powerset", file, seed)

# Print path to the saved file
print("Evaluations saved to XGBoost_model_evaluations.txt")



********** Random Seed: 42 **********


Evaluating Binary Relevance
Accuracy: 0.9567567567567568
Macro F1 Score: 0.9544115767927367
Weighted F1 score: 0.9900245802936503
Micro F1 score: 0.9902619207521827
Macro Precision: 0.9894731204068343
Weighted Precision: 0.9899461031404756
Micro Precision: 0.9899295065458208
Macro Recall: 0.9322022466888545
Weighted Recall: 0.9905945582801478
Micro Recall: 0.9905945582801478
Hamming Loss: 0.0095004095004095
Zero One Loss: 0.043243243243243246
Jaccard Similarity: 0.9802059202059202
Classification Report:
                      precision    recall  f1-score   support

   Credential Access       0.97      0.96      0.96        94
     Defence Evasion       0.99      1.00      1.00       507
           Discovery       0.99      0.99      0.99       449
           Execution       0.99      1.00      1.00       377
      Initial Access       1.00      0.57      0.73         7
         Persistance       0.99      0.99      0.99       228
        Exfilt

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Multilabel Confusion Matrix:
[[[474   2]
  [  1  78]]

 [[ 52   3]
  [  2 498]]

 [[ 95   4]
  [  1 455]]

 [[175   0]
  [  1 379]]

 [[551   0]
  [  1   3]]

 [[324   0]
  [  4 227]]

 [[536   1]
  [  2  16]]

 [[350   0]
  [  1 204]]

 [[111   2]
  [  1 441]]

 [[ 57   4]
  [  0 494]]

 [[422   3]
  [  2 128]]]




********** Random Seed: 995 **********


Evaluating Label Powerset
Accuracy: 0.9837837837837838
Macro F1 Score: 0.9773866136341854
Weighted F1 score: 0.994693312460965
Micro F1 score: 0.9947251999319381
Macro Precision: 0.9935238390081789
Weighted Precision: 0.9948999826271523
Micro Precision: 0.9948944860449285
Macro Recall: 0.9646550429037131
Weighted Recall: 0.99455597141885
Micro Recall: 0.99455597141885
Hamming Loss: 0.005077805077805078
Zero One Loss: 0.01621621621621616
Jaccard Similarity: 0.9897211497211498
Classification Report:
                      precision    recall  f1-score   support

   Credential Access       0.96      0.96      0.96        79
     Defence

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))




********** Random Seed: 98 **********


Evaluating Binary Relevance
Accuracy: 0.9531531531531532
Macro F1 Score: 0.9437689590229013
Weighted F1 score: 0.9874937000767265
Micro F1 score: 0.9876500857632933
Macro Precision: 0.9792744430486189
Weighted Precision: 0.9876344090647622
Micro Precision: 0.9876500857632933
Macro Recall: 0.9240113674471498
Weighted Recall: 0.9876500857632933
Micro Recall: 0.9876500857632933
Hamming Loss: 0.011793611793611793
Zero One Loss: 0.04684684684684681
Jaccard Similarity: 0.9714500214500215
Classification Report:
                      precision    recall  f1-score   support

   Credential Access       0.94      0.95      0.94        80
     Defence Evasion       0.99      1.00      0.99       497
           Discovery       0.99      0.99      0.99       447
           Execution       1.00      0.99      1.00       372
      Initial Access       1.00      0.50      0.67         4
         Persistance       0.99      0.99      0.99       225
        Exfil

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))




********** Random Seed: 262 **********


Evaluating Binary Relevance
Accuracy: 0.954954954954955
Macro F1 Score: 0.9625528327959647
Weighted F1 score: 0.9904793289799414
Micro F1 score: 0.990593466735078
Macro Precision: 0.9902673745720233
Weighted Precision: 0.9904211784027592
Micro Precision: 0.9904240766073872
Macro Recall: 0.9444407166212013
Weighted Recall: 0.9907629148135477
Micro Recall: 0.9907629148135477
Hamming Loss: 0.009009009009009009
Zero One Loss: 0.04504504504504503
Jaccard Similarity: 0.978103818103818
Classification Report:
                      precision    recall  f1-score   support

   Credential Access       0.96      0.94      0.95        85
     Defence Evasion       0.99      0.99      0.99       500
           Discovery       0.99      0.99      0.99       450
           Execution       0.99      1.00      0.99       374
      Initial Access       1.00      0.60      0.75         5
         Persistance       1.00      0.99      0.99       223
        Exfiltr

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))




********** Random Seed: 1600 **********


Evaluating Binary Relevance
Accuracy: 0.9441441441441442
Macro F1 Score: 0.9290386008401442
Weighted F1 score: 0.9875514073431391
Micro F1 score: 0.9876673713883017
Macro Precision: 0.9392817914034102
Weighted Precision: 0.9841890043228062
Micro Precision: 0.9841994382022472
Macro Recall: 0.9212842798503454
Weighted Recall: 0.9911598302687411
Micro Recall: 0.9911598302687411
Hamming Loss: 0.011466011466011465
Zero One Loss: 0.05585585585585584
Jaccard Similarity: 0.9727691977691979
Classification Report:
                      precision    recall  f1-score   support

   Credential Access       0.96      0.93      0.94        72
     Defence Evasion       0.99      1.00      0.99       475
           Discovery       0.98      1.00      0.99       440
           Execution       0.99      0.99      0.99       366
      Initial Access       0.50      0.50      0.50         2
         Persistance       0.98      1.00      0.99       219
        Exf